In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# hyper parameters

num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [4]:
# dataset has PILimage image of range [0,1]
# we transform them to tensers of normalized range [-1,1]

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [5]:
# CIFAR10
train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,
                                           transform=transforms.ToTensor())

test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,
                                           transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle =True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle =False)

examples = iter(train_loader)
samples, labels = examples.next()
print(samples.shape, labels.shape)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
torch.Size([4, 3, 32, 32]) torch.Size([4])


In [6]:
classes = ('airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck')

In [7]:
# implement conv net 
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [8]:
model = ConvNet().to(device)

In [9]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate) 

In [12]:
# training loop

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # 4 3 32 32 ----> 4 3 1024
    # input_layer: 3 input channels 6  o/p channals 5 kernal size

    images = images.to(device)
    labels = labels.to(device)

    # forward
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backwards
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
 
    if (i+1) % 2000 == 0:
      print(f'epoch {epoch+1} / {num_epochs}, steps {i+1}/{n_total_steps}, loos = {loss.item():.4f}')


# test

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

epoch 1 / 4, steps 2000/12500, loos = 1.7186
epoch 1 / 4, steps 4000/12500, loos = 1.5381
epoch 1 / 4, steps 6000/12500, loos = 2.2602
epoch 1 / 4, steps 8000/12500, loos = 1.8679
epoch 1 / 4, steps 10000/12500, loos = 1.9698
epoch 1 / 4, steps 12000/12500, loos = 0.9580
epoch 2 / 4, steps 2000/12500, loos = 1.7079
epoch 2 / 4, steps 4000/12500, loos = 0.8447
epoch 2 / 4, steps 6000/12500, loos = 0.9315
epoch 2 / 4, steps 8000/12500, loos = 1.3540
epoch 2 / 4, steps 10000/12500, loos = 1.4815
epoch 2 / 4, steps 12000/12500, loos = 1.7813
epoch 3 / 4, steps 2000/12500, loos = 1.0202
epoch 3 / 4, steps 4000/12500, loos = 0.6614
epoch 3 / 4, steps 6000/12500, loos = 1.3992
epoch 3 / 4, steps 8000/12500, loos = 0.6439
epoch 3 / 4, steps 10000/12500, loos = 1.9592
epoch 3 / 4, steps 12000/12500, loos = 0.6384
epoch 4 / 4, steps 2000/12500, loos = 0.4727
epoch 4 / 4, steps 4000/12500, loos = 2.2053
epoch 4 / 4, steps 6000/12500, loos = 0.8618
epoch 4 / 4, steps 8000/12500, loos = 1.5082
epoc